In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime

from scipy.stats import linregress



#Get 2019 Data
CapitalBike201901 = "../UnzipRawFiles/2019/201901-capitalbikeshare-tripdata.csv"
CapitalBike201902 = "../UnzipRawFiles/2019/201902-capitalbikeshare-tripdata.csv"
CapitalBike201903 = "../UnzipRawFiles/2019/201903-capitalbikeshare-tripdata.csv"
CapitalBike201904 = "../UnzipRawFiles/2019/201904-capitalbikeshare-tripdata.csv"
CapitalBike201905 = "../UnzipRawFiles/2019/201905-capitalbikeshare-tripdata.csv"
CapitalBike201906 = "../UnzipRawFiles/2019/201906-capitalbikeshare-tripdata.csv"
CapitalBike201907 = "../UnzipRawFiles/2019/201907-capitalbikeshare-tripdata.csv"
CapitalBike201908 = "../UnzipRawFiles/2019/201908-capitalbikeshare-tripdata.csv"
CapitalBike201909 = "../UnzipRawFiles/2019/201909-capitalbikeshare-tripdata.csv"
CapitalBike201910 = "../UnzipRawFiles/2019/201910-capitalbikeshare-tripdata.csv"
CapitalBike201911 = "../UnzipRawFiles/2019/201911-capitalbikeshare-tripdata.csv"
CapitalBike201912 = "../UnzipRawFiles/2019/201912-capitalbikeshare-tripdata.csv"



CapitalBike201901_df = pd.read_csv(CapitalBike201901)
CapitalBike201902_df = pd.read_csv(CapitalBike201902)
CapitalBike201903_df = pd.read_csv(CapitalBike201903)
CapitalBike201904_df = pd.read_csv(CapitalBike201904)
CapitalBike201905_df = pd.read_csv(CapitalBike201905)
CapitalBike201906_df = pd.read_csv(CapitalBike201906)
CapitalBike201907_df = pd.read_csv(CapitalBike201907)
CapitalBike201908_df = pd.read_csv(CapitalBike201908)
CapitalBike201909_df = pd.read_csv(CapitalBike201909)
CapitalBike201910_df = pd.read_csv(CapitalBike201910)
CapitalBike201911_df = pd.read_csv(CapitalBike201911)
CapitalBike201912_df = pd.read_csv(CapitalBike201912)

In [2]:
CapitalBike201906_df.tail()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
350139,733,2019-06-30 23:57:17,2019-07-01 00:09:30,31609,Maine Ave & 7th St SW,31610,Eastern Market / 7th & North Carolina Ave SE,W22399,Casual
350140,5127,2019-06-30 23:58:11,2019-07-01 01:23:39,31292,22nd St & Constitution Ave NW,31261,21st St & Constitution Ave NW,W20536,Casual
350141,5145,2019-06-30 23:58:24,2019-07-01 01:24:10,31292,22nd St & Constitution Ave NW,31261,21st St & Constitution Ave NW,W20531,Casual
350142,331,2019-06-30 23:59:08,2019-07-01 00:04:39,31610,Eastern Market / 7th & North Carolina Ave SE,31612,D St & Maryland Ave NE,W00795,Member
350143,467,2019-06-30 23:59:17,2019-07-01 00:07:05,31319,Wisconsin Ave & Brandywine St NW,32014,Friendship Hts Metro / Wisconsin Ave & Wiscons...,W23253,Casual


In [3]:
combined_2019_df = pd.concat([CapitalBike201901_df, CapitalBike201902_df,CapitalBike201903_df,CapitalBike201904_df, CapitalBike201905_df, CapitalBike201906_df,CapitalBike201907_df,CapitalBike201908_df,CapitalBike201909_df,CapitalBike201910_df,CapitalBike201911_df,CapitalBike201912_df], ignore_index=True)


In [4]:
combined_2019_df.count()

Duration                3398417
Start date              3398417
End date                3398417
Start station number    3398417
Start station           3398417
End station number      3398417
End station             3398417
Bike number             3398403
Member type             3398417
dtype: int64

In [5]:
#cleanup
combined_2019_df.dropna(inplace = True)

In [6]:
combined_2019_df.count()

Duration                3398403
Start date              3398403
End date                3398403
Start station number    3398403
Start station           3398403
End station number      3398403
End station             3398403
Bike number             3398403
Member type             3398403
dtype: int64

In [ ]:
combined_2019_df['DateTime'] =  pd.to_datetime(combined_2019_df['Start date'], format='%Y-%m-%d %H:%M:%S')
# extracting time from timestamp
combined_2019_df['Time'] = combined_2019_df['DateTime'].dt.strftime('%H:%M')
combined_2019_df['Month'] = combined_2019_df['DateTime'].dt.strftime('%b')
combined_2019_df['Date'] = combined_2019_df['DateTime'].dt.strftime('%x')
#combined_2019_df.tail()

In [ ]:
combined_2019_df = combined_2019_df.drop(['Start station number','Start station','End station number','End station'], axis = 1)

combined_2019_df.head()

In [ ]:

#combined_2019_df.to_csv('2019Data.csv', index = False)


In [ ]:
#get 2020 data
CapitalBike202001 = "../UnzipRawFiles/2020/202001-capitalbikeshare-tripdata.csv"
CapitalBike202002 = "../UnzipRawFiles/2020/202002-capitalbikeshare-tripdata.csv"
CapitalBike202003 = "../UnzipRawFiles/2020/202003-capitalbikeshare-tripdata.csv"
CapitalBike202005 = "../UnzipRawFiles/2020/202005-capitalbikeshare-tripdata.csv"
CapitalBike202006 = "../UnzipRawFiles/2020/202006-capitalbikeshare-tripdata.csv"
CapitalBike202007 = "../UnzipRawFiles/2020/202007-capitalbikeshare-tripdata.csv"
CapitalBike202008 = "../UnzipRawFiles/2020/202008-capitalbikeshare-tripdata.csv"
CapitalBike202009 = "../UnzipRawFiles/2020/202009-capitalbikeshare-tripdata.csv"
CapitalBike202010 = "../UnzipRawFiles/2020/202010-capitalbikeshare-tripdata.csv"
CapitalBike202011 = "../UnzipRawFiles/2020/202011-capitalbikeshare-tripdata.csv"
CapitalBike202012 = "../UnzipRawFiles/2020/202012-capitalbikeshare-tripdata.csv"


CapitalBike202001_df = pd.read_csv(CapitalBike202001)
CapitalBike202002_df = pd.read_csv(CapitalBike202002)
CapitalBike202003_df = pd.read_csv(CapitalBike202003)
CapitalBike202005_df = pd.read_csv(CapitalBike202005)
CapitalBike202006_df = pd.read_csv(CapitalBike202006)
CapitalBike202007_df = pd.read_csv(CapitalBike202007)
CapitalBike202008_df = pd.read_csv(CapitalBike202008)
CapitalBike202009_df = pd.read_csv(CapitalBike202009)
CapitalBike202010_df = pd.read_csv(CapitalBike202010)
CapitalBike202011_df = pd.read_csv(CapitalBike202011)
CapitalBike202012_df = pd.read_csv(CapitalBike202012)

In [ ]:
CapitalBike202007_df.head()

In [ ]:
#combine data
#combined_2020_df = pd.concat([CapitalBike202001_df, CapitalBike202002_df,CapitalBike202003_df, CapitalBike202005_df, CapitalBike202006_df, CapitalBike202007_df, CapitalBike202008_df, CapitalBike202009_df, CapitalBike202010_df, CapitalBike202011_df, CapitalBike202012_df], ignore_index=True)
# #get columns that are needed
#combined_2020_df = combined_2020_df[["Duration", "Start date","End date","Start station number","Start station","End station number","End station","Bike number","Member type"]]

#====Data format changed between March 2020 and May 2020....converted May-Dec dataset to align with Jan2019 - Mar2020 datasets.
combined_2020_1st_df = pd.concat([CapitalBike202001_df, CapitalBike202002_df,CapitalBike202003_df], ignore_index=True)
combined_2020_2nd_df = pd.concat([CapitalBike202005_df, CapitalBike202006_df, CapitalBike202007_df, CapitalBike202008_df, CapitalBike202009_df, CapitalBike202010_df, CapitalBike202011_df, CapitalBike202012_df], ignore_index=True)
#====Had to create a duration column in seconds from starting and ending times to alighn with 2019 data.
combined_2020_2nd_df['Duration'] = pd.to_timedelta(pd.to_datetime(combined_2020_2nd_df['ended_at'], format='%Y-%m-%d %H:%M:%S')-pd.to_datetime(combined_2020_2nd_df['started_at'], format='%Y-%m-%d %H:%M:%S')).dt.total_seconds()
combined_2020_2ndcol_df = combined_2020_2nd_df[['Duration','started_at','ended_at','start_station_id','start_station_name','end_station_id','end_station_name','ride_id','member_casual']]
combined_2020_2ndcol_df = combined_2020_2ndcol_df.rename(columns={'started_at':'Start date','ended_at':'End date','start_station_id':'Start station number','start_station_name':'Start station','end_station_id':'End station number','end_station_name':'End station','ride_id':'Bike number','member_casual':'Member type'})

In [ ]:
combined_2020_2ndcol_df.info()

In [ ]:
combined_2020_1st_df.info()

In [ ]:
#---NEW consolidated 2020 dataset
combined_2020_df = pd.concat([combined_2020_1st_df, combined_2020_2ndcol_df], ignore_index=True)
combined_2020_df = combined_2020_df.drop(['Start station number','Start station','End station number','End station'], axis = 1)
combined_2020_df.tail()

In [ ]:
#combined_2020_df.dropna(inplace = True)
#No clean up needed as count matches
#combined_2020_df.dropna(inplace = True)

In [ ]:
combined_2020_df.count()


#combined_2020_df.to_csv('2020Data.csv', index = False)

In [ ]:
#Cleanup and create data
combined_2020_df['DateTime'] =  pd.to_datetime(combined_2020_df['Start date'], format='%Y-%m-%d %H:%M:%S')
# extracting time from timestamp
combined_2020_df['Time'] = combined_2020_df['DateTime'].dt.strftime('%H:%M')
combined_2020_df['Month'] = combined_2020_df['DateTime'].dt.strftime('%b')
combined_2020_df['Date'] = combined_2020_df['DateTime'].dt.strftime('%x')
combined_2020_df.head()

In [ ]:
combined_2020_df.head()

In [ ]:
combined_2019_df = combined_2019_df.astype({'Duration': float})
combined_2019_df.head()

In [ ]:
#combined_2020_NeededColumns_df.to_csv('2020Data.csv', index = False)

In [ ]:
months = ['Jan', 'Feb', 'Mar', 'Apr','May','Jun', 'Jul', 'Aug','Sep', 'Oct', 'Nov', 'Dec']

sum_2019 = combined_2019_df['Month'].value_counts()
sum_2019 =sum_2019.reindex(months) 
#sum_2019

In [ ]:
sum_2020 = combined_2020_df['Month'].value_counts()
sum_2020 = pd.concat([sum_2020.iloc[:1], pd.Series(0, ['Apr']),sum_2020[1:]])
sum_2020 =sum_2020.reindex(months) 
#sum_2020

In [ ]:

combined_2020_df['Member type'].replace(to_replace ="casual",
                 value ="Casual",inplace = True)

combined_2020_df['Member type'].replace(to_replace ="member",
                 value ="Member",inplace = True)

In [ ]:
combined_2019_df["DayName"] = combined_2019_df["DateTime"].dt.day_name()
combined_2019_df["DayNum"] = combined_2019_df["DateTime"].dt.weekday

outlierIndex = combined_2019_df[combined_2019_df["Duration"] 
                                > (combined_2019_df["Duration"].mean() + (2 * (combined_2019_df["Duration"].std())))].index

combined_2019_df.drop(outlierIndex, inplace=True)

sample_2019_df = combined_2019_df.sample(200)
sample_2019_df["DayInYear"] = sample_2019_df["DateTime"].dt.strftime('%j')
sorted_sample_df = sample_2019_df.sort_values(by="Date")
sorted_sample_df.sort_values(by="DayNum", inplace=True)
sorted_sample_df.tail()

In [ ]:
#months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
x_values = sorted_sample_df['DateTime']
y_values = sorted_sample_df['DayName']
duration = sorted_sample_df['Duration']
fig = plt.figure(figsize=(12,8))
ax = fig.add_axes([.1,.1,.8,.8])
ax.scatter(x_values, y_values, s=duration, c="red", alpha=0.5, edgecolor="black")
ax.set_xlabel('Month')
ax.set_ylabel('Weekday')
#ax.xaxis_date()
#ax.set_xticks(np.arange(1,12,1))
#ax.set_xticklabels(months)

plt.show()

In [ ]:
#---Line Chart, ALL rides by month Year over Year
 

plt.plot(sum_2019, Label = '2019')
plt.plot(sum_2020, Label = '2020')

plt.title("City Bike Share 2019 v 2020")
plt.xlabel("Month")
plt.ylabel("Bike Share Rides")
 
plt.legend()
plt.show()

In [ ]:
# Bar Graph month over month

x = np.arange(len(months))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, sum_2019, width, label='2019')
rects2 = ax.bar(x + width/2, sum_2020, width, label='2020')

ax.set_ylabel('Bike Rides')
ax.set_title('City Bike Share 2019 v 2020 month over month')
ax.set_xticks(x)
ax.set_xticklabels(months)
ax.legend()


fig.tight_layout()

plt.show()

In [ ]:
time_2019_df = combined_2019_df
time_2019_df["Time"] = pd.to_datetime(time_2019_df['Time'], format='%H:%M').dt.hour

bins = [-1,5,7,9,11,13,15,17,19,21,24]

bin_labels = ["Pre-Dawn","Early Morning", "Morning Rush", "Mid Morning", "Lunch Rush", "Early Afternoon", "Mid Afternoon", "Evening Rush", "Late Evening", "Overnight"]

# Apply bins and labels
time_df_2019 = time_2019_df
time_df_2019["Time of Day"] = pd.cut(time_df_2019["Time"], bins, labels=bin_labels)

# Group time data frame
time_df_2019 = time_df_2019.groupby('Time of Day').count()

time_2019_counts = time_df_2019["Time"]

time_2019_sum = pd.DataFrame({"Number of Rides": time_2019_counts})
time_2019_sum

In [ ]:
time_2020_df = combined_2020_df
time_2020_df["Time"] = pd.to_datetime(time_2020_df['Time'], format='%H:%M').dt.hour

bins = [-1,5,7,9,11,13,15,17,19,21,24]

bin_labels = ["Pre-Dawn","Early Morning", "Morning Rush", "Mid Morning", "Lunch Rush", "Early Afternoon", "Mid Afternoon", "Evening Rush", "Late Evening", "Overnight"]

# Apply bins and labels
time_df_2020 = time_2020_df
time_df_2020["Time of Day"] = pd.cut(time_df_2020["Time"], bins, labels=bin_labels)

# Group time data frame
time_df_2020 = time_df_2020.groupby('Time of Day').count()

time_2020_counts = time_df_2020["Time"]

time_2020_sum = pd.DataFrame({"Number of Rides": time_2020_counts})
time_2020_sum